<a href="https://colab.research.google.com/github/Vinh-thuy/data_valorization/blob/main/demo/prompt_generator_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Connexion Google Drive {display-mode: "form"}

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/MyDrive/P4T/google_colab/demo/


Mounted at /content/gdrive/
/content/gdrive/MyDrive/P4T/google_colab/demo


In [156]:
import pandas as pd 
review_summary=pd.read_csv('/content/gdrive/MyDrive/P4T/google_colab/demo/review_summary.csv')

In [6]:
#@title Construction de la connexion à la base PostgreSQL {display-mode: "form"}

from postgres import postgresQuery
from utilities import DB_HOSTNAME,DB_PORT, DB_NAME, DB_USER, DB_PWD, PG_COL_TO_CLEAN_TABLE_RESTAURANT
pg = postgresQuery(DB_HOSTNAME, DB_PORT, DB_NAME, DB_USER, DB_PWD)

In [16]:
#@title Récupération des tables : restaurant {display-mode: "form"}
city = 'Paris'

sql_query_1 = "select *  from restaurant r2  where r2.adress_city  = '" + city + "'"
sql_query_2 = "select r1.*, r2.review_id as review_id, r2.reviews_data as reviews_data from restaurant r1, review r2  where r1.restaurant_id = r2.restaurant_id and r1.adress_city  = '" + city + "'"


liste_restaurant = pg.query_db(sql_query_1)
liste_restaurant_review = pg.query_db(sql_query_2)


liste_restaurant['created_at'] = pd.to_datetime(liste_restaurant['created_at']).dt.normalize()

liste_restaurant_review['created_at'] = pd.to_datetime(liste_restaurant_review['created_at']).dt.normalize()

In [19]:
mapping_restaurant_review = liste_restaurant_review[['review_id', 'restaurant_id', 'criteria']]

In [43]:
mapping_restaurant_review_criteriaNotNull = mapping_restaurant_review[mapping_restaurant_review.review_id.isin(mapping_restaurant_review.dropna(how='any',axis=0).review_id)]

In [47]:
mapping_restaurant_review_criteriaNotNull.criteria.iloc[0]

"['Services de restauration : Desserts', 'Offre : Bière', 'Services de restauration : Dîner', 'Services de restauration : Déjeuner', 'Services disponibles : Repas sur place', 'Offre : Produits sains', 'Services : Toilettes', 'Offre : Cafés', 'Offre : Alcools', 'Services de restauration : Traiteur', 'Ambiance : Décontracté', 'Offre : Cocktails et apéritifs', 'Services disponibles : Livraison', 'Offre : Vin', 'Ambiance : Chaleureux', 'Offre : Spiritueux', 'Planning : Réservations acceptées', 'Accessibilité : Parking accessible en fauteuil roulant']"

In [116]:
#@title Load GPT Extraction NLP CLOUD depuis Google Drive : review_summary_split {display-mode: "form"}

import pandas as pd 
review_summary_split=pd.read_csv('/content/gdrive/MyDrive/P4T/google_colab/demo/review_summary_split.csv')  
review_summary_split.head(1)

,Unnamed: 0,keyphrase_extract_review,review_id,review_category,created_at
0,0,"['1', ' Pâtes fraîches délicieuses et savoureu...",66347,review_positif,2023-01-10 09:04:17


In [119]:
df = review_summary_split
df2 = mapping_restaurant_review_criteriaNotNull

df3 = df.merge(df2, on='review_id', how='left')
df3.head(1)

,Unnamed: 0,keyphrase_extract_review,review_id,review_category,created_at,restaurant_id,criteria
0,0,"['1', ' Pâtes fraîches délicieuses et savoureu...",66347,review_positif,2023-01-10 09:04:17,835,"['Offre : Cocktails et apéritifs', 'Services :..."


In [320]:
#@title Fonction : clean_text (text) {display-mode: "form"}

def clean_text(text):
  import re

  #text = re.sub(r'\n\d', ' ', text).replace("\nspoilers","").replace("\n","")
  cleanedtext = []
  for item in nlp(text):
      if not item.is_stop:
          cleanedtext.append(item.text)
  text = ' '.join(cleanedtext)

  #text = re.sub(r"\'", ' ', text).replace("\nspoilers","").replace("\n","")
  #text = re.sub(r" . ", ' ', text).replace("","").replace("","")
  #text = re.sub(r",. ", ', ', text).replace("","").replace("","")

  lst = text.split(",") 
  
  return lst


In [120]:
df3.keyphrase_extract_review.iloc[0]

"['1', ' Pâtes fraîches délicieuses et savoureuses', '  ', ' Portions généreuses', '  ', ' Tiramisu à tomber', '  ', ' Service dynamique et souriant', '  ', ' Garniture généreuse et variée sur les pizzas', '  ', ' Accueil agréable.']"

In [121]:
clean_text(df3.keyphrase_extract_review.iloc[3])

['[Pizza napolitaine excellente',
 'Service excellent',
 'Pâtes carbonara excellentes',
 'Ambiance bienvenue',
 'Nourriture italienne rafraîchissante  ]']

In [131]:
clean_text(df3.keyphrase_extract_review.iloc[3])

['[ Pizza napolitaine excellente ',
 ' Service excellent ',
 ' Pâtes carbonara excellentes ',
 ' Ambiance bienvenue ',
 ' Nourriture italienne rafraîchissante   ]']

In [134]:
df = review_summary
df2 = mapping_restaurant_review_criteriaNotNull
df3 = df.merge(df2, on='review_id', how='left')
df3['kpe_list'] = ""
df3['criteria_list'] = ""

In [135]:
lst_kpe = clean_text(df3.keyphrase_extract_review.iloc[3])

In [136]:
df3.kpe_list.append(lst_kpe)

TypeError: ignored

In [185]:
df3.restaurant_id.unique()

array([835, 792, 774, 977, 744, 805, 790, 819, 925, 927, 906, 912, 833,
       907, 985])

In [186]:
df4 = pd.DataFrame()

In [187]:

 
# Create the pandas DataFrame with column name is provided explicitly
df4 = pd.DataFrame(df3.restaurant_id.unique(), columns=['restaurant_id'])
 

In [417]:
print(df5[df5.restaurant_id == 792].kpe_list.values[0])

['1. Service professionnel et rapide' ' . Pizza de qualité' ' . Prix raisonnables' ' . Très bonne sélection de bières' ' . Ambiance agréable' ' . Emplacement pratique' ' . Personnel aimable et serviable' ' . Options adaptées aux régimes alimentaires spécifiques' ' . Choix variés dans les plats' ' 0. Décor accueillant et confortable.']['1' ' Ambiance conviviale' '  ' ' Variete des plats offerts' '  ' ' Qualite des plats offerts' '  ' ' Prix raisonnables' '  ' ' Service rapide et attentionne' '  ' ' Personnels sympathiques' '  ' ' Agencement adequat' '  ' ' Endroit propre et presentable' '  ' ' Menus saisonniers interessants.']['1. Une ambiance agréable et chaleureuse ' '  . Des plats délicieux et respectant bien la tradition culinaire ' '  . Un personnel accueillant et serviable ' '  . Des tarifs abordables ' '  . Une carte variée et intéressante ' '  . Une décoration originale et soignée ' '  . Des menus variés offrant une bonne sélection de plats ' '  . Un bon équilibre entre le prix 

In [409]:
df = review_summary
df2 = mapping_restaurant_review_criteriaNotNull
df3 = df.merge(df2, on='review_id', how='left')

df4 = pd.DataFrame(df3.restaurant_id.unique(), columns=['restaurant_id'])
df4['criteria_list'] = "[]"

df5 = df3[['restaurant_id', 'criteria']]
df5 = df5.drop_duplicates()



for (id) in enumerate(df5.restaurant_id):
  kpe_list = ""

  for keyphrase_extract_review in zip(df3[df3.restaurant_id == id[1]].keyphrase_extract_review):
    for elt in enumerate(keyphrase_extract_review[0].split(",")):
      kpe_list = kpe_list + elt[1]
  df5.loc[df5["restaurant_id"] == id[1], "kpe_list"] = [kpe_list]


'\nfor (id) in enumerate(df5.restaurant_id):\n  criteria_list = []\n\n  for criteria in zip(df5[df5.restaurant_id == id[1]].criteria):\n    for elt in enumerate(criteria[0].split(",")):\n      criteria_list.append(elt)\n  df5.loc[df5["restaurant_id"] == id[1], "criteria_list"] = [criteria_list]\n'

In [410]:
df5

,restaurant_id,criteria,kpe_list
0,835,"['Offre : Cocktails et apéritifs', 'Services :...",['1. Le service est rapide et amical' ' . La q...
1,792,"['Offre : Cocktails et apéritifs', 'Paiements ...",['1. Service professionnel et rapide' ' . Pizz...
5,774,"['Services de restauration : Places assises', ...",['1. Prix raisonnables' ' . Super bon' ' . Sup...
6,977,"['Offre : Cocktails et apéritifs', 'Services :...",['1. Garçons sympathiques' ' . service rapide...
8,744,"['Offre : Cocktails et apéritifs', 'Paiements ...",['1. Plats variés adaptés au menu' ' . Nourri...
11,805,['Services disponibles : Livraison sans contac...,['1. Produits de qualité' ' . Tout est fait ma...
13,790,"['Offre : Cocktails et apéritifs', 'Paiements ...",['1. Service très long' ' . Ambiance sympathi...
16,819,"['Offre : Cocktails et apéritifs', 'Services :...",['1' ' Ambiance agréable ' ' ' ' Personnel sy...
21,925,"['Offre : Cocktails et apéritifs', 'Paiements ...",['1. Une cuisine italienne traditionnelle et i...
24,927,"['Offre : Cocktails et apéritifs', 'Services :...",['1. La pizza est savoureuse et bien cuite' ' ...


In [359]:
df5[df5.restaurant_id == 792].criteria[1]

'[\'Offre : Cocktails et apéritifs\', \'Paiements : Paiements mobiles NFC\', \'Services : Toilettes\', \'Paiements : Cartes de paiement\', \'Offre : Plats végétaliens\', \'Services de restauration : Dîner\', \'Ambiance : Décontracté\', \'Services de restauration : Brunch\', \'Paiements : Cartes de crédit\', \'Services : Bar disponible sur place\', \'Services de restauration : Places assises\', \'Offre : Bière\', \'Services : Chaises hautes\', \'Offre : Convient aux végétariens\', \'Accessibilité : Entrée accessible en fauteuil roulant\', \'Services de restauration : Petit déjeuner\', \'Services disponibles : Repas sur place\', \'Services de restauration : Desserts\', \'Services : Convient aux enfants\', \'Accessibilité : Parking accessible en fauteuil roulant\', \'Offre : Spiritueux\', \'Offre : Vin\', \'Services de restauration : Traiteur\', \'Services disponibles : Livraison\', \'Services disponibles : Terrasse\', \'Offre : Service de restauration ouvert tard en soirée\', \'Offre : R

In [351]:
df5[df5.restaurant_id == 792].kpe_list[1]

[(0, "['1. Service professionnel et rapide'"),
 (1, " ' . Pizza de qualité'"),
 (2, " ' . Prix raisonnables'"),
 (3, " ' . Très bonne sélection de bières'"),
 (4, " ' . Ambiance agréable'"),
 (5, " ' . Emplacement pratique'"),
 (6, " ' . Personnel aimable et serviable'"),
 (7, " ' . Options adaptées aux régimes alimentaires spécifiques'"),
 (8, " ' . Choix variés dans les plats'"),
 (9, " ' 0. Décor accueillant et confortable.']"),
 (0, "['1'"),
 (1, " ' Ambiance conviviale'"),
 (2, " '  '"),
 (3, " ' Variete des plats offerts'"),
 (4, " '  '"),
 (5, " ' Qualite des plats offerts'"),
 (6, " '  '"),
 (7, " ' Prix raisonnables'"),
 (8, " '  '"),
 (9, " ' Service rapide et attentionne'"),
 (10, " '  '"),
 (11, " ' Personnels sympathiques'"),
 (12, " '  '"),
 (13, " ' Agencement adequat'"),
 (14, " '  '"),
 (15, " ' Endroit propre et presentable'"),
 (16, " '  '"),
 (17, " ' Menus saisonniers interessants.']"),
 (0, "['1. Une ambiance agréable et chaleureuse '"),
 (1, " '  . Des plats déli

In [352]:
for keyphrase_extract_review in zip(df3[df3.restaurant_id == 792].keyphrase_extract_review):
  print(keyphrase_extract_review)

("['1. Service professionnel et rapide', ' . Pizza de qualité', ' . Prix raisonnables', ' . Très bonne sélection de bières', ' . Ambiance agréable', ' . Emplacement pratique', ' . Personnel aimable et serviable', ' . Options adaptées aux régimes alimentaires spécifiques', ' . Choix variés dans les plats', ' 0. Décor accueillant et confortable.']",)
("['1', ' Ambiance conviviale', '  ', ' Variete des plats offerts', '  ', ' Qualite des plats offerts', '  ', ' Prix raisonnables', '  ', ' Service rapide et attentionne', '  ', ' Personnels sympathiques', '  ', ' Agencement adequat', '  ', ' Endroit propre et presentable', '  ', ' Menus saisonniers interessants.']",)
("['1. Une ambiance agréable et chaleureuse ', '  . Des plats délicieux et respectant bien la tradition culinaire ', '  . Un personnel accueillant et serviable ', '  . Des tarifs abordables ', '  . Une carte variée et intéressante ', '  . Une décoration originale et soignée ', '  . Des menus variés offrant une bonne sélection d

In [271]:
type(kpe_list)

list

In [272]:
df5

,restaurant_id,criteria
0,835,"['Offre : Cocktails et apéritifs', 'Services :..."
1,792,"['Offre : Cocktails et apéritifs', 'Paiements ..."
5,774,"['Services de restauration : Places assises', ..."
6,977,"['Offre : Cocktails et apéritifs', 'Services :..."
8,744,"['Offre : Cocktails et apéritifs', 'Paiements ..."
11,805,['Services disponibles : Livraison sans contac...
13,790,"['Offre : Cocktails et apéritifs', 'Paiements ..."
16,819,"['Offre : Cocktails et apéritifs', 'Services :..."
21,925,"['Offre : Cocktails et apéritifs', 'Paiements ..."
24,927,"['Offre : Cocktails et apéritifs', 'Services :..."


In [153]:
def create_criteria_list(df):


SyntaxError: ignored

In [139]:
for id in enumerate(df3.restaurant_id):
  for (keyphrase_extract_review, criteria) in zip(df3[df3.restaurant_id == id[1]].keyphrase_extract_review, df3[df3.restaurant_id == id[1]].criteria):
    kpe_list = kpe_list.append(clean_text(keyphrase_extract_review))
    criteria_list = criteria_list.append(clean_text(criteria))
return kpe_list, criteria_list

0     ['1. Le service est rapide et amical', ' . La ...
2     ['1.Cuisine italienne généreuse', '  .Personne...
9     ['1. Un service excellent', ' . Nourriture exc...
29    ['1. Ambiance agréable', '  . Cuisine apprécié...
36                                       ['Excellent!']
49    ['1. Excellente qualité des produits', '  . Pl...
Name: keyphrase_extract_review, dtype: object

In [175]:
df = review_summary
df2 = mapping_restaurant_review_criteriaNotNull
df3 = df.merge(df2, on='review_id', how='left')
kpe_list = []
criteria_list = []

for i, id in enumerate(df3.restaurant_id):
  for (keyphrase_extract_review, criteria) in zip(df3[df3.restaurant_id == id].keyphrase_extract_review, df3[df3.restaurant_id == id].criteria):
    kpe_list.append(clean_text(keyphrase_extract_review))  
    #criteria_list = criteria_list.append(clean_text(criteria))

In [182]:
id = 835
kpe_list = []
for (keyphrase_extract_review, criteria) in zip(df3[df3.restaurant_id == id].keyphrase_extract_review, df3[df3.restaurant_id == id].criteria):
  kpe_list.append(clean_text(keyphrase_extract_review))  
  #criteria_list = criteria_list.append(clean_text(criteria))

In [184]:
kpe_list

[['[ 1 service rapide amical ',
  ' . qualité pizzas excellente ',
  ' . atmosphère amusante ',
  ' . Grande variété pizzas ',
  ' . Reservation recommandée   ]'],
 ['[ 1.Cuisine italienne généreuse ',
  '   .Personnel agréable disponible rush ',
  ' .Lieu agréable   ]'],
 ['[ 1 service excellent ',
  ' . Nourriture excellente ',
  ' . Sentir maison ',
  ' . Pizza napolitaine ',
  ' . Pâtes carbonara   ]'],
 ['[ 1 Ambiance agréable ',
  '   Cuisine appréciée ',
  ' . Cadre confortable ',
  '   Personnel sympathique ',
  ' . Plats variés copieux   ]'],
 ['[ Excellent   ]'],
 ['[ 1 Excellente qualité produits ',
  '   Plat abondant quantité bien présenté ',
  '   Excellent rapport qualité prix ',
  '   Ambiance agréable atmosphère chaleureuse ',
  '   Professionnalisme employés ',
  '   Equipe souriante serviable ',
  '   Excellent service clientèle ',
  '   Bonne variété plats ',
  '   Vins qualité exceptionnelle ',
  '   . Table bien placée joliment présentée   ]']]

In [ ]:
"""
avis positif du restaurant : proche du centre ville. Burger tres bon. le patron est super sympa. service tres rapide pour le diner. allergene bien indiqué
avis négatif du restaurant : le restaurant est trop petit. le restaurant n'est pas adapté au handicapé. 
avis sur les plats : plat végétarien proposé. salade est sublime. les pates sont super. le gateau à la fraise n'est vraiment pas bon
services disponible du restaurant : 
"""